In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import RidgeCV
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV


pd.set_option('display.float_format', lambda x: '%.3f' % x)

xl = pd.ExcelFile("Project_Data.xlsx")
xl.sheet_names
df = xl.parse("Sheet1")

In [2]:
df.head(2)
df.describe()
print df.corr()

                             ASX Lit Share  ASX Dark Share  ASX On Market  \
ASX Lit Share                        1.000           0.093          0.956   
ASX Dark Share                       0.093           1.000          0.125   
ASX On Market                        0.956           0.125          1.000   
ASX Continous Share                  0.903           0.074          0.849   
ASX Lit                              0.146          -0.201          0.109   
Chi-X Lit                           -0.580          -0.224         -0.584   
ASX Centre Point                    -0.190           0.093         -0.184   
Chi-X Dark                          -0.112          -0.139         -0.330   
Reported to ASX                      0.165          -0.063          0.156   
Reported to Chi-X                   -0.046          -0.058         -0.088   
Market                               0.067          -0.176          0.027   
Lit + Dark                           0.956           0.124          1.000   

In [3]:
# Project, bring in the file for analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import RidgeCV
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV


pd.set_option('display.float_format', lambda x: '%.3f' % x)

xl = pd.ExcelFile("Project_Data.xlsx")
xl.sheet_names
df = xl.parse("Sheet1")

#preview of the raw data and return the column names
df.head()
df.columns
# examine to see if there are any null rows
df.isnull().sum()
#there are rows with null data, let us remove these rows
df = df.dropna()
# we have 547 rows of data left, enough for the regression. the dataframe has been cleaned.
df.count()

#drop columns which have the dates
df = df.drop('Date', axis=1)
df = df.drop('Date.1', axis=1)

df.columns
#The data is all in differrent units: percentages, raw numbers, $'s, basis points. 
#If we want to analyse the relative effects of each independent variable on the dependent variables, 
#we'll need to normalise all the data.




Index([u'ASX Lit Share', u'ASX Dark Share', u'ASX On Market',
       u'ASX Continous Share', u'ASX Lit', u'Chi-X Lit', u'ASX Centre Point',
       u'Chi-X Dark', u'Reported to ASX', u'Reported to Chi-X', u'Market',
       u'Lit + Dark', u'Lit', u'Reports', u'CP Share', u'Row',
       u'Lit Trading - Vanilla', u'Trade Registration', u'Block Discovery',
       u'Undisclosed', u'CentrePoint', u'Icebergs', u'Pref', u'UCP',
       u'CXA Market', u'Specials', u'NBBO Trade Report', u'Portfolio Specials',
       u'Late Trades', u'Broker Pref', u'20-Day Realised SPI Vol', u'MOC',
       u'Spread Differrential', u'NBBO Trade Report.1', u'%Sweep of Lit',
       u'Index Rebalance', u'ASX Lit Share (20 MA)', u'ASX Dark Share (20 MA)',
       u'ASX On Market.1', u'ASX Continous Share (20 MA)',
       u'Number of Customer Meetings', u'Average Meetings',
       u'Sensitive Announcments'],
      dtype='object')

In [4]:
# the first regression we will look at is for the ASX Lit Market Share. For this, we will focus on the following columns of our raw dataset

#'ASX Lit Share', 'Lit Trading - Vanilla', 'Block Discovery', 'Undisclosed', 'Icebergs', 'Late Trades', '20-Day Realised SPI Vol', 'MOC', 'Spread Differrential', '%Sweep of Lit', 'Index Rebalance', u'ASX Lit Share (20 MA)', 'Average Meetings'  

dfLitShare = df[['ASX Lit Share', 'Chi-X Lit','ASX Centre Point',
       'Chi-X Dark', 'Reported to ASX', 'Reported to Chi-X',
       'Lit Trading - Vanilla', 'Block Discovery',
       'Undisclosed', 'CentrePoint', 'Icebergs', 'Pref', 'UCP',
       'Late Trades', 'Broker Pref', '20-Day Realised SPI Vol', 'MOC',
       'Spread Differrential', 'NBBO Trade Report.1', '%Sweep of Lit',
       'Index Rebalance', 'ASX Dark Share (20 MA)']]
#['ASX Lit Share', 'Lit Trading - Vanilla', 'Block Discovery', '20-Day Realised SPI Vol', 'Spread Differrential', '%Sweep of Lit', 'Index Rebalance', u'ASX Lit Share (20 MA)']]
dfLitShare

#The data is all in differrent units: percentages, raw numbers, $'s, basis points. 
#If we want to analyse the relative effects of each independent variable on the dependent variables, 
#we'll need to normalise all the data.

std_scale = preprocessing.StandardScaler().fit(dfLitShare)
dfLitShare_std = std_scale.transform(dfLitShare)

ColumnNames = ['ASXLitShare', 'Chi-X Lit','ASX Centre Point',
       'Chi-X Dark', 'Reported to ASX', 'Reported to Chi-X',
       'Lit Trading - Vanilla', 'Block Discovery',
       'Undisclosed', 'CentrePoint', 'Icebergs', 'Pref', 'UCP',
       'Late Trades', 'Broker Pref', '20-Day Realised SPI Vol', 'MOC',
       'Spread Differrential', 'NBBO Trade Report.1', '%Sweep of Lit',
       'Index Rebalance', 'ASX Dark Share (20 MA)']

dfLitShare_std = pd.DataFrame(dfLitShare_std, columns = ColumnNames) 
dfLitShare_std

#none of the independent variables look extremely correllated so we donot have to drop any independent variables
dfLitShare_std.corr()

#plot the data to see how it looks
#pd.scatter_matrix(dfLitShare_std, figsize=(15,15))
#plt.show()

#dfLitShare_std.describe()

#let us start the regression. We have chosen Lasso Ridge & Decision Trees for our regression. For those features which are
#irrellevant, we want the coefficients to be 0 rather than very close to zero like ridge regression

X = dfLitShare_std.iloc[:, 1:] #selection of columns, all the columns except the last one.
y = dfLitShare_std.iloc[:,0]


from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


#use LassoCV to select best alpha, 100 partitions of the data

from sklearn.linear_model import LassoCV
alpha_range = 0.01**np.arange(-3, 3)
lascv = LassoCV(normalize=True, alphas=alpha_range, cv =50)
lascv.fit(X_train, y_train)
lascv.alpha_
lascv.coef_
preds = lascv.predict(X_test)
#print lascv.coef_
print 'RMSE (Lasso CV reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds))
print ('')
print pd.DataFrame(zip(X_train.columns, lascv.coef_)).sort_values(by=1, ascending=False)
print ('')

#use RidgeCV to select best alpha, 100 partitions of the data

alpha_range = 10.**np.arange(-3, 3)
#CV = indicates cross validation
rregcv = RidgeCV(normalize=True, scoring='mean_squared_error', alphas=alpha_range, cv =50)
rregcv.fit(X_train, y_train)
#print rregcv.coef_
rregcv.alpha_
preds = rregcv.predict(X_test)
print 'RMSE (Ridge CV reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds))
print ('')
print pd.DataFrame(zip(X_train.columns, rregcv.coef_)).sort_values(by=1, ascending=False)
print ('')


#Decision Tree

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
rtree = tree.DecisionTreeRegressor()
rtree.fit(X_train, y_train)
scores = cross_val_score(rtree, X, y, cv=50, scoring='mean_squared_error')
mse_scores = scores


print 'RMSE (Decision Tree) =', mse_scores.mean()
print ('')
features = X_train.columns.tolist()
print pd.DataFrame(zip(features, rtree.feature_importances_)).sort_values(by=1, ascending=False)
print ('')


#'ASX Lit Share', 'ASX Dark Share', 'ASX On Market',
#       'ASX Continous Share', 'ASX Lit', 'Chi-X Lit', 'ASX Centre Point',
#       'Chi-X Dark', 'Reported to ASX', 'Reported to Chi-X', 'Market',
#       'Lit + Dark', 'Lit', 'Reports', 'CP Share', 'Row',
#       'Lit Trading - Vanilla', 'Trade Registration', 'Block Discovery',
#       'Undisclosed', 'CentrePoint', 'Icebergs', 'Pref', 'UCP',
#       'CXA Market', 'Specials', 'NBBO Trade Report', 'Portfolio Specials',
#       'Late Trades', 'Broker Pref', '20-Day Realised SPI Vol', 'MOC',
#       'Spread Differrential', 'NBBO Trade Report.1', '%Sweep of Lit',
#       'Index Rebalance', 'ASX Lit Share (20 MA)', 'ASX Dark Share (20 MA)'

RMSE (Lasso CV reg.) = 0.259561518996

                          0      1
5     Lit Trading - Vanilla  0.896
6           Block Discovery  0.438
20   ASX Dark Share (20 MA)  0.059
11                      UCP  0.028
14  20-Day Realised SPI Vol  0.025
4         Reported to Chi-X  0.015
17      NBBO Trade Report.1  0.012
1          ASX Centre Point  0.007
12              Late Trades  0.006
8               CentrePoint  0.001
13              Broker Pref  0.000
10                     Pref  0.000
3           Reported to ASX  0.000
15                      MOC -0.000
2                Chi-X Dark -0.004
9                  Icebergs -0.008
7               Undisclosed -0.015
19          Index Rebalance -0.017
18            %Sweep of Lit -0.035
16     Spread Differrential -0.056
0                 Chi-X Lit -1.354

RMSE (Ridge CV reg.) = 0.259827457071

                          0      1
5     Lit Trading - Vanilla  0.886
6           Block Discovery  0.439
20   ASX Dark Share (20 MA)  0.064
14  20-Day 